In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

import src.data_cleaning as dc

In [2]:
df = dc.create_df('bigml_59c28831336c6604c800002a.csv')
X = df.drop('churn', axis=1)
y = df['churn']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
class Dataprep(BaseEstimator):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        
        df = X.copy()
        df['international plan'] = df['international plan'] == 'yes'
        df['voice mail plan'] = df['voice mail plan'] == 'yes'
    
        df['account months'] = np.ceil(df['account length']/30).astype(int)
        df['total minutes'] = df['total day minutes']+df['total eve minutes']+df['total night minutes']+df['total intl minutes']
        df['total calls'] = df['total day calls']+df['total eve calls']+df['total night calls']+df['total intl calls']
        df['total cost'] = df['total day charge']+df['total eve charge']+df['total night charge']+df['total intl charge']
        df['cost per day'] = df['total cost']/df['account length']
    
        df.drop(['state', 'phone number', 'area code'], axis=1, inplace=True)
    
        return df

In [5]:
rfc = RandomForestClassifier(random_state=42, n_estimators=100)

pipe = Pipeline(steps=[
    ("dataprep", Dataprep()),
    ("scaler", StandardScaler()),
    ("model", rfc)
])

In [6]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('dataprep', Dataprep()),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
                                        verbose=0, warm_start=False))],
   

In [7]:
confusion_matrix(y_train, pipe.predict(X_train))

array([[2141,    0],
       [   0,  358]])

In [8]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

       False       0.98      1.00      0.99       709
        True       1.00      0.86      0.93       125

    accuracy                           0.98       834
   macro avg       0.99      0.93      0.96       834
weighted avg       0.98      0.98      0.98       834



In [10]:
roc_auc_score(y_train, pipe.predict(X_train))

1.0